In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers
from tensorflow.keras import layers, optimizers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l2

In [4]:
# Import data
class_names_dict = np.load("class_names.npy", allow_pickle=True).item()
attributes_npy = np.load("attributes.npy", allow_pickle=True)
attributes_df = pd.read_csv("attributes.txt", header=None, index_col = 0, sep=" |::", names=("attribute", "value"), engine='python')
training_labels_df = pd.read_csv("train_images.csv", header = 0)
test_images_paths = pd.read_csv("test_images_path.csv", header = 0)
test_images_sample = pd.read_csv("test_images_sample.csv", header = 0)

In [5]:
# local paths 
training_dir = "train_images_inferred_augmented/train_images" #This is the inferred augmented dir
testing_dir = "test_images" 

## Load Training

In [12]:
# This loads the training images from the balanced augmented training dir

#because we load all images in one batch kernel crashes 
#adjust this parameter accordingly (50 is max)
num_of_images_to_load = 50


images = []
labels = []

resize_dim = (224, 224)

# Iterate through subdirectories
for subdir in range(1, 201):
    subdir_path = os.path.join(training_dir, str(subdir))
    
    counter = 0
    # Iterate through files in each subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith(".jpg"):
            
            # Read images
            image_path = os.path.join(subdir_path, filename)
            img = cv2.imread(image_path)
            
            # Fix images
            img_resized = cv2.resize(img, resize_dim)
            img_normalized = img_resized / 255.0
            
            #Load images
            images.append(img_normalized)
            labels.append(subdir)
            
            counter += 1
            if counter >= num_of_images_to_load:
                break

# Convert lists to numpy arrays
images_array = np.array(images)
labels_array = np.array(labels)


In [13]:
len(labels_array)

600

## Train Model

In [14]:
# Split to training and val set

train_birds_set, val_birds_set, train_labels_set, val_labels_set = train_test_split(images, labels, test_size=0.2, random_state=42)

In [15]:
# Convert sets to arrays because models dont accept lists

train_birds_set = np.array(train_birds_set)
val_birds_set = np.array(val_birds_set)
train_labels_set = np.array(train_labels_set)
val_labels_set = np.array(val_labels_set)

In [16]:
#Build MODEL 

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-1:]:
    layer.trainable = True
    


def create_model():
    model = models.Sequential()
    model.add(base_model)
    
    # Add L2 regularization
    model.add(layers.GlobalAveragePooling2D())
    #model.add(layers.Dropout(0.2))
    model.add(layers.Dense(256, activation='relu',))
    model.add(layers.Dense(201, activation='softmax'))
    #model.add(layers.Dense(201, activation='softmax',))
    return model

# Create a fresh instance of the model
model = create_model()


2023-12-14 15:09:13.814502: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


58889256/58889256 [==============================] - 8s 0us/step


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 201)               51657     
                                                                 
Total params: 14,897,673
Trainable params: 182,985
Non-trainable params: 14,714,688
_________________________________________________________________


In [18]:
# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(

    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    vertical_flip=True,
)

# Fit the ImageDataGenerator on the training data
datagen.fit(train_birds_set)

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model.fit(train_birds_set, train_labels_set, epochs=20, 
                    validation_data=(val_birds_set, val_labels_set))

Epoch 1/20


/Users/stathis/opt/anaconda3/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


 3/15 [=====>........................] - ETA: 1:15 - loss: 5.4230 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [56]:
# In case the above model is good SAVE IT !!

#model.save('best_model_33.h5')

In [7]:
model = tf.keras.models.load_model('best_model_33.h5')

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

## Test Model

In [6]:
## Read images from test_dir (resize and rescale) in order and load test_images[] array


resize_dim = (224, 224)

def read_image(image_path):
    image = cv2.imread(testing_dir+image_path)
    img_resized = cv2.resize(image, resize_dim)
    img_normalized = img_resized / 255.0
    return img_normalized


test_images_paths['pixel_array'] = test_images_paths['image_path'].apply(read_image)

test_images = np.array(test_images_paths['pixel_array'].tolist())


KeyboardInterrupt: 

In [28]:
predictions = model.predict(test_images)

125/125 [==============================] - 2s 14ms/step


In [29]:
predicted_labels = np.argmax(predictions, axis=1)

In [30]:
predicted_labels

array([ 67, 119,  74, ...,  78, 187, 108])

In [31]:
test_images_sample['label'] = predicted_labels

In [32]:
test_images_sample

,id,label
0,1,67
1,2,119
2,3,74
3,4,12
4,5,73
...,...,...
3995,3996,68
3996,3997,75
3997,3998,78
3998,3999,187


In [33]:
csv_file_path = 'kaggle_file.csv'


test_images_sample.to_csv(csv_file_path, index=False)


print(f'DataFrame has been saved to {csv_file_path}')


DataFrame has been saved to kaggle_file.csv
